# Template - Backtesting

### Import Library

In [95]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [96]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [97]:
data = dataset.copy()

In [98]:
data = data[data.index > '2020-11-01 00:00:00']
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-10-25 09:20:00,1338.4,1338.6,1337.4,1337.7,4128
2024-10-25 09:25:00,1337.7,1338.0,1337.0,1337.7,2796
2024-10-25 09:30:00,1337.7,1338.0,1335.5,1336.2,4931


In [99]:
NO_SIGNAL = ''
LONG_SIGNAL = 'long'
SHORT_SIGNAL = 'short'
CLOSE_LONG_SIGNAL = 'close_long'
CLOSE_SHORT_SIGNAL = 'close_short'
CLOSE_SIGNAL = 'close_all'
#
SL = 3
TP = 7

In [100]:
def prepare_data(data):
    bbands = ta.bbands(data.Close, length=200, std=2)
    bbands.rename(columns={'BBL_200_2.0': 'BBL', 'BBU_200_2.0': 'BBU'}, inplace=True)
    data = data.join(bbands[['BBL', 'BBU']])
    data['middle_band'] = data['Close'].rolling(200).mean()
    data['color'] = data.apply(
        lambda r: 'doji' if r['Open'] == r['Close'] else (
            'green' if r['Open'] < r['Close'] else 'red'), axis=1)
    data['prev_color'] = data['color'].shift(1)
    data['prev_close'] = data['Close'].shift(1)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    return data
    
def cal_signal(row):
    signal = NO_SIGNAL
    if 100 * row.name.hour + row.name.minute == 905:
        if row['color'] == 'red' and row['prev_color'] == 'red' and row['Close'] > row['middle_band'] + 2 * (row['BBU'] - row['middle_band']) / 3:
            signal = SHORT_SIGNAL
        elif row['color'] == 'red' and row['Close'] > row['BBU'] and row['Close'] < row['prev_close']:
            signal = SHORT_SIGNAL
    elif 100 * row.name.hour + row.name.minute == 1430:
        signal = CLOSE_SIGNAL
    else:
        if 100 * row.name.hour + row.name.minute < 1400:
            if row['color'] == 'red' and row['prev_color'] == 'red' and row['Close'] < row['BBU'] and row['prev_close'] > row['BBU']:
                signal = SHORT_SIGNAL
            elif row['color'] == 'red' and row['prev_color'] == 'red' and row['Close'] < row['middle_band'] and row['prev_close'] > row['middle_band']:
                signal = SHORT_SIGNAL
    return signal

In [101]:
class BollingerBandS(Strategy):
    def init(self):
        return

    def next(self):
        signal = self.data.signal[-1]
        price = self.data.Close[-1]
        if self.position and signal == CLOSE_SIGNAL:
            self.position.close()
            return
        if (self.position.is_long and signal == CLOSE_LONG_SIGNAL) or (self.position.is_short and signal == CLOSE_SHORT_SIGNAL):
            self.position.close()
            return

        if signal == LONG_SIGNAL:
            sl = price - SL
            tp = price + TP
            self.buy(sl=sl, tp=tp, size=1)
        elif signal == SHORT_SIGNAL:
            sl = price + SL
            tp = price - TP
            self.sell(sl=sl, tp=tp, size=1)

In [102]:
prepared_data = prepare_data(data)
prepared_data

,Open,High,Low,Close,Volume,BBL,BBU,middle_band,color,prev_color,prev_close,signal
Date,,,,,,,,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910,NaN,NaN,NaN,doji,None,NaN,
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670,NaN,NaN,NaN,red,doji,900.1,
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329,NaN,NaN,NaN,doji,red,899.4,
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722,NaN,NaN,NaN,red,doji,899.5,
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,NaN,NaN,NaN,red,red,898.6,
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25 09:20:00,1338.4,1338.6,1337.4,1337.7,4128,1340.365492,1370.723508,1355.5445,red,green,1338.5,
2024-10-25 09:25:00,1337.7,1338.0,1337.0,1337.7,2796,1340.084399,1370.727601,1355.4060,doji,red,1337.7,
2024-10-25 09:30:00,1337.7,1338.0,1335.5,1336.2,4931,1339.764990,1370.757010,1355.2610,red,doji,1337.7,


In [103]:
bt = Backtest(prepared_data, BollingerBandS, commission=.003, exclusive_orders=False)
stats = bt.run()
# bt.plot()
print(stats)

Start                     2020-11-02 09:00:00
End                       2024-10-25 09:40:00
Duration                   1453 days 00:40:00
Exposure Time [%]                    9.339911
Equity Final [$]                    8601.0387
Equity Peak [$]                    10003.0718
Return [%]                         -13.989613
Buy & Hold Return [%]               48.405733
Return (Ann.) [%]                   -3.744856
Volatility (Ann.) [%]                0.669294
Sharpe Ratio                        -5.595233
Sortino Ratio                       -5.278283
Calmar Ratio                        -0.264082
Max. Drawdown [%]                  -14.180642
Avg. Drawdown [%]                   -4.742974
Max. Drawdown Duration     1448 days 20:10:00
Avg. Drawdown Duration      482 days 23:37:00
# Trades                                  399
Win Rate [%]                        29.824561
Best Trade [%]                       0.468587
Worst Trade [%]                     -0.717164
Avg. Trade [%]                    

In [104]:
pd.set_option('display.expand_frame_repr', False)
print(stats['_trades'])

     Size  EntryBar  ExitBar  EntryPrice  ExitPrice     PnL  ReturnPct           EntryTime            ExitTime   Tag        Duration
0      -1       225      250    906.6718      912.4 -5.7282  -0.006318 2020-11-06 10:45:00 2020-11-06 14:15:00  None 0 days 03:30:00
1      -1       399      401    911.5571      917.3 -5.7429  -0.006300 2020-11-11 13:55:00 2020-11-11 14:05:00  None 0 days 00:10:00
2      -1       387      405    915.7445      921.8 -6.0555  -0.006613 2020-11-11 11:30:00 2020-11-11 14:25:00  None 0 days 02:55:00
3      -1       445      448    918.3367      924.2 -5.8633  -0.006385 2020-11-12 13:30:00 2020-11-12 13:45:00  None 0 days 00:15:00
4      -1       523      534    935.6845      931.3  4.3845   0.004686 2020-11-16 10:15:00 2020-11-16 11:10:00  None 0 days 00:55:00
..    ...       ...      ...         ...        ...     ...        ...                 ...                 ...   ...             ...
394    -1     50067    50085   1343.4575     1350.6 -7.1425  -0.00531

In [107]:
negative_trades = stats['_trades'][stats._trades.PnL < 0]
print(negative_trades)

     Size  EntryBar  ExitBar  EntryPrice  ExitPrice     PnL  ReturnPct           EntryTime            ExitTime   Tag        Duration
0      -1       225      250    906.6718      912.4 -5.7282  -0.006318 2020-11-06 10:45:00 2020-11-06 14:15:00  None 0 days 03:30:00
1      -1       399      401    911.5571      917.3 -5.7429  -0.006300 2020-11-11 13:55:00 2020-11-11 14:05:00  None 0 days 00:10:00
2      -1       387      405    915.7445      921.8 -6.0555  -0.006613 2020-11-11 11:30:00 2020-11-11 14:25:00  None 0 days 02:55:00
3      -1       445      448    918.3367      924.2 -5.8633  -0.006385 2020-11-12 13:30:00 2020-11-12 13:45:00  None 0 days 00:15:00
5      -1       714      762    947.8479      952.4 -4.5521  -0.004803 2020-11-20 09:10:00 2020-11-20 14:45:00  None 0 days 05:35:00
..    ...       ...      ...         ...        ...     ...        ...                 ...                 ...   ...             ...
387    -1     49634    49635   1353.7266     1360.8 -7.0734  -0.00522